In [1]:
%load_ext autoreload
%autoreload 2
from manage_datasets import load_features_outcomes, load_features_labels, make_training_sets, make_stacked_sets, load_feature_list
from model_utils import make_survival_model, save_model
import yaml

device = 'cmod'
#dataset_path = 'random_2000_shots_50%_flattop'
#dataset_path = 'no_ufo_flattop_1452_shots_50%_disruptive'
#dataset_path = 'random_flattop_256_shots_60%_disruptive'
dataset_path = 'no_ufo_flattop_7736_shots_6%_disruptive'

# Make training sets if they haven't been created yet
try:
    numeric_feats = load_feature_list(device, dataset_path)
except:
    make_training_sets(device, dataset_path, random_seed=0)
    numeric_feats = load_feature_list(device, dataset_path)
    

In [3]:
# Load training, test, validation sets
x_train, y_train = load_features_outcomes(device, dataset_path, 'train', numeric_feats)
x_val, y_val = load_features_outcomes(device, dataset_path, 'val', numeric_feats)

# Create labels for binary classifier models according to the disruptive window
disruptive_window = 0.4 # seconds before disruption labeled as 'disruptive'
_, labels_train = load_features_labels(device, dataset_path, 'train', disruptive_window, numeric_feats)
_, labels_val = load_features_labels(device, dataset_path, 'val', disruptive_window, numeric_feats)

In [9]:
# Retrain the best found models for this dataset
model_strings = ['cph', 'dsm']

for model_string in model_strings:
    with open(f"models/{device}/{dataset_path}/{model_string}.yaml", 'r') as stream:
        config = yaml.safe_load(stream)
        for key, value in config.items():
            # Check if value is not an integer
            if not isinstance(value, int):
                config[key] = value['value']

    model = make_survival_model(config)
    model.fit(x_train, y_train)
    save_model(model, model_string, device, dataset_path, numeric_feats)

Saved model to models/cmod/no_ufo_flattop_7736_shots_6%_disruptive/cph.pkl


 36%|███▌      | 18/50 [00:24<00:44,  1.39s/it]

Saved model to models/cmod/no_ufo_flattop_7736_shots_6%_disruptive/dsm.pkl


In [14]:
# Train a random forest model and save it
dprf_model = run_rf_model(x_train, x_val, labels_train, labels_val)
save_model(dprf_model, transformer, f'dprf_{int(disruptive_window*1000)}_ms', device, dataset_path, numeric_feats)

Saved model to models/cmod/no_ufo_flattop_1452_shots_50%_disruptive/dprf_400_ms.pkl


In [7]:
# Train a dcph model and save it
dcph_model = run_survival_model('dcph', x_train, x_val, outcomes_train, outcomes_val)
save_model(dcph_model, transformer, 'dcph', device, dataset_path, numeric_feats)

100%|██████████| 50/50 [00:16<00:00,  2.99it/s]
c:\Users\zkeith\Documents\Risk-Aware Frameworks\disruption-survival-analysis\.venv\lib\site-packages\sksurv\linear_model\coxph.py:127: RuntimeWarning: overflow encountered in power
  y=np.power(self.baseline_survival_.y, risk_score[i]))


Error in fitting model for parameters: {'bs': 50, 'epochs': 50, 'layers': [100, 100], 'learning_rate': 0.001}


100%|██████████| 50/50 [00:16<00:00,  2.97it/s]
c:\Users\zkeith\Documents\Risk-Aware Frameworks\disruption-survival-analysis\.venv\lib\site-packages\sksurv\linear_model\coxph.py:127: RuntimeWarning: overflow encountered in power
  y=np.power(self.baseline_survival_.y, risk_score[i]))


Error in fitting model for parameters: {'bs': 50, 'epochs': 50, 'layers': [100, 100], 'learning_rate': 0.0001}


100%|██████████| 50/50 [00:16<00:00,  2.97it/s]
c:\Users\zkeith\Documents\Risk-Aware Frameworks\disruption-survival-analysis\.venv\lib\site-packages\sksurv\linear_model\coxph.py:127: RuntimeWarning: overflow encountered in power
  y=np.power(self.baseline_survival_.y, risk_score[i]))


Error in fitting model for parameters: {'bs': 100, 'epochs': 50, 'layers': [100, 100], 'learning_rate': 0.001}


100%|██████████| 50/50 [00:16<00:00,  2.96it/s]
c:\Users\zkeith\Documents\Risk-Aware Frameworks\disruption-survival-analysis\.venv\lib\site-packages\sksurv\linear_model\coxph.py:127: RuntimeWarning: overflow encountered in power
  y=np.power(self.baseline_survival_.y, risk_score[i]))


Error in fitting model for parameters: {'bs': 100, 'epochs': 50, 'layers': [100, 100], 'learning_rate': 0.0001}
Saved model to models/cmod/no_ufo_flattop_1452_shots_50%_disruptive/stack_2/dcph.pkl


In [ ]:
# Train a random survival forest model and save it
#rsf_model = run_survival_model('rsf', x_train, x_val, outcomes_train, outcomes_val)
#save_model(rsf_model, transformer, 'rsf', device, dataset_path, numeric_feats)


Saved model to models/rsf_cmod_random_256_shots_60%_flattop.pkl


In [8]:
%%capture --no-display
# Train a dsm model and save it
dsm_model = run_survival_model('dsm', x_train, x_val, outcomes_train, outcomes_val)
save_model(dsm_model, transformer, 'dsm', device, dataset_path, numeric_feats)

In [3]:
%%capture --no-display
# Train a dcm model and save it
dcm_model = run_survival_model('dcm', x_train, x_val, outcomes_train, outcomes_val)
save_model(dcm_model, transformer, 'dcm', device, dataset_path, numeric_feats)

: 

: 

In [ ]:
# Train a temporal cph model and save it
#k = 10
##x_train_temporal, outcomes_train_temporal = stack_features(x_train, outcomes_train, k)
#x_val_temporal, outcomes_val_temporal = stack_features(x_val, outcomes_val, k)
#cph_temporal_model = run_survival_model('cph', x_train_temporal, x_val_temporal, outcomes_train_temporal, outcomes_val_temporal, selection=selection)
#save_model(cph_temporal_model, transformer, f'cph_temporal_{k}k', device, dataset)

In [2]:
# Make a temporal dataset
stack_size = 2
make_stacked_sets(device, dataset_path, 'train', stack_size)
make_stacked_sets(device, dataset_path, 'test', stack_size)
make_stacked_sets(device, dataset_path, 'val', stack_size)